# Process data for E_vs_r_scan calculation

This Notebook is designed for reading and converting results from the E_vs_r_scan calculation

Library imports

In [1]:
import glob
import os
from collections import OrderedDict
from datetime import date
from math import floor

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from DataModelDict import DataModelDict as DM

import atomman as am
import atomman.unitconvert as uc
import iprPy

## 1. Raw Data

This section reads in or generates the raw_data associated with the calculation. 

### 1.1 Parameters

__lib_directory__ is the path to the data model record library to use 

In [2]:
lib_directory = 'C:\\Users\\lmh1\\Documents\\calculations\\ipr\\library_2016_10_26'

__raw_csv__ gives the location of the csv file for raw_data

In [3]:
raw_csv = os.path.join(lib_directory, 'E_vs_r_scan_raw.csv')

__build_raw__ indicates if raw_data is built from lib_directory or read in from raw_csv.
    
- = True -- Read data from all refine_structure_static records and save data to raw_csv

- = False -- Read data from raw_csv

In [4]:
build_raw = True

### 1.2 Conversion Functions

Functions used in converting a tree-like data model representation into a flattened dictionary are listed here. Flattened dictionaries are necessary for building the pandas DataFrame of all the data.

__model_2_dict__ takes a data model (as a DataModelDict) and converts it into an equivalent flat dictionary.

In [5]:
def model_2_dict(model):
    """Convert a structured data model for the calculation into an equivalent flat dictionary."""
    
    values = {}
    
    try: calculation = model.find('calculation')
    except: pass
    else:
        try: values['calc_key'] =               calculation['id']
        except: pass
        try: values['calc_type'] =              calculation['script']
        except: pass
        try: values['strain_range'] =           calculation[['run-parameter', 'strain-range']]
        except: pass
        try: values['load_options'] =           calculation[['run-parameter', 'load_options']]
        except: pass
        try: values['size_mult_a_lo'] =         calculation[['run-parameter', 'size-multipliers', 'a', 0]]
        except: pass
        try: values['size_mult_a_hi'] =         calculation[['run-parameter', 'size-multipliers', 'a', 1]]
        except: pass
        try: values['size_mult_b_lo'] =         calculation[['run-parameter', 'size-multipliers', 'b', 0]]
        except: pass
        try: values['size_mult_b_hi'] =         calculation[['run-parameter', 'size-multipliers', 'b', 1]]
        except: pass
        try: values['size_mult_c_lo'] =         calculation[['run-parameter', 'size-multipliers', 'c', 0]]
        except: pass
        try: values['size_mult_c_hi'] =         calculation[['run-parameter', 'size-multipliers', 'c', 1]]
        except: pass
        try: values['minimum_r'] =              calculation[['run-parameter', 'minimum_r']]
        except: pass
        try: values['maximum_r'] =              calculation[['run-parameter', 'maximum_r']]
        except: pass
        try: values['number_of_steps_r'] =      calculation[['run-parameter', 'number_of_steps_r']]
        except: pass
    
    try: potential = model.find('potential')
    except: pass
    else:
        try: values['pot_key'] =                potential['key']
        except: pass
        try: values['pot_id'] =                 potential['id']
        except: pass

    try: system_info = model.find('system-info')
    except: pass
    else:
        try: values['load_file'] =              system_info[['artifact', 'file']]
        except: pass
        try: values['load_style'] =             system_info[['artifact', 'format']]
        except: pass
        try: values['prototype'] =              system_info[['artifact', 'family']]
        except: pass
        try: values['symbols'] =                '-'.join(system_info['symbols'])
        except: pass
    
    try: cohesive_energy_relation = model.find('cohesive-energy-relation')
    except: pass 
    else:
        try: values['plot_r'] =                uc.value_unit(cohesive_energy_relation['r'])
        except: pass
        try: values['plot_a'] =                uc.value_unit(cohesive_energy_relation['a'])
        except: pass
        try: values['plot_Ecoh'] =             uc.value_unit(cohesive_energy_relation['cohesive-energy'])
        except: pass
    
    try: phase_state = model.find('phase-state')
    except: pass
    else:    
        try: values['temperature'] =            uc.value_unit(phase_state['temperature'])
        except: pass
        try: values['pressure_xx'] =            uc.value_unit(phase_state['pressure-xx'])
        except: pass
        try: values['pressure_yy'] =            uc.value_unit(phase_state['pressure-yy'])
        except: pass
        try: values['pressure_zz'] =            uc.value_unit(phase_state['pressure-zz'])
        except: pass
        
    try: system = model.find('relaxed-atomic-system')
    except: pass
    else:        
        try: values['relaxed-system-a'] =       uc.value_unit(system['cell'].find('a'))
        except: pass
        try: values['relaxed-system-b'] =       uc.value_unit(system['cell'].find('b'))
        except: pass
        try: values['relaxed-system-c'] =       uc.value_unit(system['cell'].find('c'))
        except: pass
        try: values['relaxed-system-alpha'] =   system['cell'].find('alpha')
        except: pass
        try: values['relaxed-system-beta'] =    system['cell'].find('beta')
        except: pass
        try: values['relaxed-system-gamma'] =   system['cell'].find('gamma')
        except: pass
        
        ucell, symbols = am.load('system_model', model, key='relaxed-atomic-system')
        atype_count = np.bincount(ucell.atoms_prop(key='atype'))
        values['composition'] = comp_refine(symbols, atype_count)
        
    try: system = model.finds('minimum-atomic-system')[0]
    except: pass
    else:        
        try: values['minimum-system-a'] =       uc.value_unit(system['cell'].find('a'))
        except: pass
        try: values['minimum-system-b'] =       uc.value_unit(system['cell'].find('b'))
        except: pass
        try: values['minimum-system-c'] =       uc.value_unit(system['cell'].find('c'))
        except: pass
        try: values['minimum-system-alpha'] =   system['cell'].find('alpha')
        except: pass
        try: values['minimum-system-beta'] =    system['cell'].find('beta')
        except: pass
        try: values['minimum-system-gamma'] =   system['cell'].find('gamma')
        except: pass
        
        ucell, symbols = am.load('system_model', model, key='minimum-atomic-system')
        atype_count = np.bincount(ucell.atoms_prop(key='atype'))
        values['composition'] = comp_refine(symbols, atype_count)
        
    try: values['E_cohesive'] =                 uc.value_unit(model.find('cohesive-energy'))
    except: pass    
    
    try: elastic_constants = model.find('elastic-constants')
    except: pass
    else: 
        try: values['C11'] =                    uc.value_unit(elastic_constants.find('C', yes={'ij':'1 1'})['stiffness'])
        except: pass
        try: values['C22'] =                    uc.value_unit(elastic_constants.find('C', yes={'ij':'2 2'})['stiffness'])
        except: pass
        try: values['C33'] =                    uc.value_unit(elastic_constants.find('C', yes={'ij':'3 3'})['stiffness'])
        except: pass
        try: values['C12'] =                    uc.value_unit(elastic_constants.find('C', yes={'ij':'1 2'})['stiffness'])
        except: pass
        try: values['C13'] =                    uc.value_unit(elastic_constants.find('C', yes={'ij':'1 3'})['stiffness'])
        except: pass
        try: values['C23'] =                    uc.value_unit(elastic_constants.find('C', yes={'ij':'2 3'})['stiffness'])
        except: pass
        try: values['C44'] =                    uc.value_unit(elastic_constants.find('C', yes={'ij':'4 4'})['stiffness'])
        except: pass
        try: values['C55'] =                    uc.value_unit(elastic_constants.find('C', yes={'ij':'5 5'})['stiffness'])
        except: pass
        try: values['C66'] =                    uc.value_unit(elastic_constants.find('C', yes={'ij':'6 6'})['stiffness'])
        except: pass
    
    try: values['error'] = model.find('error')
    except: pass
    
    
    return values

__comp_refine__ takes a list of symbols and count of how many times each symbol appears in a structure and generates a composition string.

In [6]:
def comp_refine(symbols, counts):
    """Takes a list of symbols and count of how many times each symbol appears and generates a composition string."""
    primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47]
    
    sym_dict = {}
    for i in xrange(len(symbols)):
        sym_dict[symbols[i]] = counts[i+1]
    
    for prime in primes:
        if max(sym_dict.values()) < prime:
            break
        
        while True:
            breaktime = False
            for value in sym_dict.values():
                if value % prime != 0:
                    breaktime = True
                    break
            if breaktime:
                break
            for key in sym_dict:
                sym_dict[key] /= prime
    
    composition=''
    for key in sorted(sym_dict):
        if sym_dict[key] > 0:
            composition += key
            if sym_dict[key] != 1:
                composition += str(sym_dict[key])
            
    return composition       

### 1.3 Code

Create raw_data

In [7]:
if build_raw:
    raw_data = []
    for fname in glob.iglob(os.path.join(lib_directory, '*', '*', '*', 'E_vs_r_scan', '*')):

        if os.path.splitext(fname)[1].lower() in ('.xml', '.json'):
            with open(fname) as f:
                model = DM(f)
            raw_data.append(model_2_dict(model))
    raw_data = pd.DataFrame(raw_data)
    raw_data.to_csv(raw_csv, index=False, float_format='%.13g')  

else:
    raw_data = pd.read_csv(raw_csv)

Show raw_data

In [8]:
raw_data

,E_cohesive,calc_key,calc_type,composition,error,load_file,load_style,maximum_r,minimum-system-a,minimum-system-b,...,pot_id,pot_key,prototype,size_mult_a_hi,size_mult_a_lo,size_mult_b_hi,size_mult_b_lo,size_mult_c_hi,size_mult_c_lo,symbols
0,"[1.09457044867, 0.208921516168, -0.52975441287...",70a5c60c-4628-4fa8-93bc-e945a7d2d015,calc_E_vs_r_scan,Cu,NaN,A1--Cu--fcc.json,system_model,6.0,3.628386,NaN,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,A1--Cu--fcc,3,0,3,0,3,0,C-u
1,"[-2.31677934795, -2.65777292449, -2.9241460642...",474dc8fb-61ec-4403-bfe1-d5ec0e3e6f65,calc_E_vs_r_scan,Cu,NaN,A15--beta-W.json,system_model,6.0,4.565657,NaN,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,A15--beta-W,3,0,3,0,3,0,C-u
2,"[-0.0624904639915, -0.7756405862, -1.376606794...",ca1db1ea-4ea5-4467-b611-37a0b5dddd83,calc_E_vs_r_scan,Cu,NaN,A2--W--bcc.json,system_model,6.0,2.869256,NaN,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,A2--W--bcc,3,0,3,0,3,0,C-u
3,"[1.09651086397, 0.210437178643, -0.52846455906...",c5c9fcc9-8a2e-41c6-a239-b636c6f07f47,calc_E_vs_r_scan,Cu,NaN,A3'--alpha-La--double-hcp.json,system_model,6.0,2.565657,4.443848,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,A3'--alpha-La--double-hcp,3,0,3,0,3,0,C-u
4,"[1.09820307818, 0.211952841119, -0.52717470525...",f6e0a610-81ea-4e66-9733-55f822d9e890,calc_E_vs_r_scan,Cu,NaN,A3--Mg--hcp.json,system_model,6.0,2.565657,4.443848,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,A3--Mg--hcp,3,0,3,0,3,0,C-u
5,"[-1.52713251163, -1.80536126006, -2.0240234936...",8310750b-2f56-492c-9a96-e20b4cfb0c2d,calc_E_vs_r_scan,Cu,NaN,A4--C--dc.json,system_model,6.0,5.365275,NaN,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,A4--C--dc,3,0,3,0,3,0,C-u
6,"[-1.55947581285, -1.99070260807, -2.3395794542...",f13a9d79-c5bf-410e-8adc-89ae8635a71c,calc_E_vs_r_scan,Cu,NaN,A5--beta-Sn.json,system_model,6.0,4.558062,NaN,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,A5--beta-Sn,3,0,3,0,3,0,C-u
7,"[0.238026078914, -0.517441579884, -1.156868682...",c8086776-cee5-4608-aa8c-5829a0a2a103,calc_E_vs_r_scan,Cu,NaN,A6--In--bct.json,system_model,6.0,2.525253,NaN,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,A6--In--bct,3,0,3,0,3,0,C-u
8,"[-2.20320407065, -2.42292947343, -2.5861308888...",adfb2ac0-043c-4b81-a79d-1d88f6bb7a11,calc_E_vs_r_scan,Cu,NaN,A7--alpha-As.json,system_model,6.0,3.350763,5.803692,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,A7--alpha-As,3,0,3,0,3,0,C-u
9,"[-1.16856106403, -1.63837379441, -2.0247922144...",e8d202b0-fe1f-4f20-99fb-c1b75fc2bf6f,calc_E_vs_r_scan,Cu,NaN,Ah--alpha-Po--sc.json,system_model,6.0,2.404040,NaN,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,Ah--alpha-Po--sc,3,0,3,0,3,0,C-u


### 1.4 Check errors

In [10]:
if 'error' in raw_data:
    for error in np.unique(raw_data[pd.notnull(raw_data.error)].error):
        print error

2008--Chenoweth-K--C-H-O
2015--Thompson-A-P--Ta
2016--Zhang-P--Ti-O


## 2. Processed Data

This section reads in or generates data that has been processed from raw_data. 

- Simulations are excluded based on an ignore_list

- Data columns are added/excluded/sorted based on a list

- Values are converted to appropriate units

- Column headers changed slightly

### 2.1 Parameters

__data_csv__ gives the location of the csv file for the processed data.

In [11]:
data_csv =   os.path.join(lib_directory, 'E_vs_r_scan.csv')

__ignore_txt__ gives the location of the file containing ignore_list, which is a list of simulations by key to exclude.

In [12]:
ignore_txt = os.path.join(lib_directory, 'E_vs_r_scan_ignore.txt')

__build_ignore_list__ indicates if ignore_list should be built based on values or read in from ignore_file. 

- = True -- Generate ignore_list from values of raw_data and save to ignore_file

- = False -- Read data in from ignore_file

In [13]:
build_ignore_list = True

__build_data__ indicates if data is built from raw_data and the ignore list, or read in from data_csv.
    
- = True -- Process raw_data and save to data_csv

- = False -- Read data in from data_csv

In [14]:
build_data = True

### 2.2 Data conversion parameters

__headers__ gives the list of data columns from raw_data to include in and how they should be renamed in data.

In [15]:
headers = OrderedDict([
        ('calc_key',    'key'),
        ('pot_id',      'potential'),
        ('prototype',   'prototype'),
        ('composition', 'composition'),
        ('plot_Ecoh',   'Ecoh (eV)'),
        ('plot_a',      'a (A)'),
        ('plot_r',      'r (A)') ])

__units__ specifies the units that any numerical values should be converted to.

In [16]:
units = {'a (A)':     'angstrom',
         'r (A)':     'angstrom',
         'Ecoh (eV)': 'eV'}

### 2.3. Code

Create ignore_list

In [17]:
if build_ignore_list:
    ignore_list = []
    
    #Add simulations with errors to ignore_list
    if 'error' in raw_data:
        ignore_list.extend(list(raw_data.calc_key[~pd.isnull(raw_data.error)]))
    
    #Add false compounds to ignore_list
    ignore_list.extend(raw_data.calc_key[raw_data['symbols'].apply(lambda x: len(np.unique(x.split('-')))) != 
                       raw_data['symbols'].apply(lambda x: len(x.split('-')))])
    
    #Add bct to ignore_list as it usually relaxes 
    #ignore_list.extend(raw_data.calc_key[raw_data['prototype'] == 'A6--In--bct'])    
    
    #Add duplicate compounds to ignore_list
    for i in xrange(len(raw_data)):
        trunc = raw_data.iloc[i+1:]
        matches = list(trunc[(trunc.pot_id ==    raw_data.iloc[i].pot_id) & 
                             (trunc.prototype == raw_data.iloc[i].prototype) &
                             (trunc.composition == raw_data.iloc[i].composition)].calc_key) 
        ignore_list.extend(matches)
    
    #Add simulations without compositions (taken from energy mins) to ignore_list 
    ignore_list.extend(list(raw_data.calc_key[pd.isnull(raw_data.composition)]))
    
    #Save ignore_list to ignore_txt
    ignore_list = np.unique(ignore_list)
    with open(ignore_txt, 'w') as f:
        f.write('\n'.join(ignore_list))
        
else:
    with open(ignore_txt) as f:
        ignore_list = f.read().split()

Process data

In [18]:
if build_data:
    #Extract only columns listed by headers' keys
    data = pd.DataFrame(raw_data, columns=headers.keys())
    
    #Rename according to headers' values
    data.rename(columns=headers, inplace=True) 
    
    #Remove entries with simulation keys in the ignore list
    data = data[~data.key.isin(ignore_list)]
    data.reset_index(drop=True, inplace=True)

    #Perform unit conversions
    for column, unit in units.iteritems():
        data[column] = uc.get_in_units(data[column], unit)
    
    data.to_csv(data_csv, index=False, float_format='%.13g')

else:
    data = pd.read_csv(data_csv)

Show data

In [19]:
data

,key,potential,prototype,composition,Ecoh (eV),a (A),r (A)
0,3e71ffb5-9e63-4804-9b90-f2425275a353,1985--Foiles-S-M--Ni-Cu,A1--Cu--fcc,Cu,"[1.09457044867, 0.782192483631, 0.489128094443...","[2.82842712475, 2.84734636973, 2.86626561471, ...","[2.0, 2.01337792642, 2.02675585284, 2.04013377..."
1,f251afef-da4f-4b3f-8403-ccd404d79ac0,1985--Foiles-S-M--Ni-Cu,A15--beta-W,Cu,"[-2.31677934795, -2.4386107152, -2.55146613718...","[4.0, 4.02675585284, 4.05351170569, 4.08026755...","[2.0, 2.01337792642, 2.02675585284, 2.04013377..."
2,93f03993-aa18-4eb3-b02d-e199c3d8b500,1985--Foiles-S-M--Ni-Cu,A2--W--bcc,Cu,"[-0.0624904639915, -0.311911499693, -0.5479900...","[2.30940107676, 2.3248485756, 2.34029607444, 2...","[2.0, 2.01337792642, 2.02675585284, 2.04013377..."
3,bebe4532-6a18-4be2-9a82-04176e3645e8,1985--Foiles-S-M--Ni-Cu,A3'--alpha-La--double-hcp,Cu,"[1.09651086397, 0.783908263896, 0.490700318931...","[2.0, 2.01337792642, 2.02675585284, 2.04013377...","[2.0, 2.01337792642, 2.02675585284, 2.04013377..."
4,53740268-5ee0-4472-b2db-d962a99fd8fb,1985--Foiles-S-M--Ni-Cu,A3--Mg--hcp,Cu,"[1.09820307818, 0.785538870317, 0.49227254342,...","[2.0, 2.01337792642, 2.02675585284, 2.04013377...","[2.0, 2.01337792642, 2.02675585284, 2.04013377..."
5,413bca4e-d89d-419d-a26b-43add0d0c87d,1985--Foiles-S-M--Ni-Cu,A4--C--dc,Cu,"[-1.52713251163, -1.62642758716, -1.7185015283...","[4.61880215352, 4.6496971512, 4.68059214888, 4...","[2.0, 2.01337792642, 2.02675585284, 2.04013377..."
6,d2149771-fe25-4599-ad69-323682252284,1985--Foiles-S-M--Ni-Cu,A5--beta-Sn,Cu,"[-1.55947581285, -1.71205953339, -1.8547969145...","[3.85682157122, 3.88261970882, 3.90841784642, ...","[2.0, 2.01337792642, 2.02675585284, 2.04013377..."
7,d7f99e91-174f-4a15-bdab-d1f7e3b8c3c4,1985--Foiles-S-M--Ni-Cu,A6--In--bct,Cu,"[0.238026078914, -0.0259334253419, -0.27597279...","[2.0, 2.01337792642, 2.02675585284, 2.04013377...","[2.0, 2.01337792642, 2.02675585284, 2.04013377..."
8,750d0d3c-d2f9-49c5-b380-09281b7a0901,1985--Foiles-S-M--Ni-Cu,A7--alpha-As,Cu,"[-2.20320407065, -2.28272967456, -2.3555839072...","[2.98851869374, 3.00850878534, 3.02849887693, ...","[2.0, 2.01337792642, 2.02675585284, 2.04013377..."
9,2538e526-03d9-45a3-b0cd-b396e8c50ee4,1985--Foiles-S-M--Ni-Cu,Ah--alpha-Po--sc,Cu,"[-1.16856106403, -1.33348893557, -1.4889749938...","[2.0, 2.01337792642, 2.02675585284, 2.04013377...","[2.0, 2.01337792642, 2.02675585284, 2.04013377..."


## 3. Text Data Table Generation

This section takes the processed data and generates per_potential txt files tabulating Ecoh vs. r data.

### 3.1 Parameters

In [20]:
per_potential_directory = 'C:\\Users\\lmh1\\Documents\\website\\per_potential'

### 3.2 Other content

Here is where additional content of the resulting html file is collected.

__data_info__ gives the file's description information prior to the data. Keywords to be replaced with generated values are surrounded by the delimiters '<' and '>'.

In [21]:
data_info = """#Cohesive energy (eV) vs. nearest neighbor radial distance (A) for various crystal structures
#Data presented for interatomic potential <potential> and composition <composition>

#NOTE: These values are for static, unrelaxed structures and use the ideal b/a and c/a ratios for the crystal structure, not the potential-specific values

#Calculations from the NIST Interatomic Potential Repository Project
#http://www.ctcms.nist.gov/potentials/

#Table generated <day>
"""

### 3.3 Code

In [22]:
#Loop over all potentials
for potential in np.unique(data.potential):
    potential_data = data[data.potential==potential]
    
    #Check that a directory exists for the potential
    if not os.path.isdir(os.path.join(per_potential_directory, potential)):
        os.makedirs(os.path.join(per_potential_directory, potential))
        
    #Loop over all compositions
    for composition in np.unique(potential_data.composition):
        composition_data = potential_data[potential_data.composition==composition]
        
        #Start data_txt by filling in data_info
        info_dict = {'potential':   potential,
                     'composition': composition,
                     'day':         str(date.today())}
        data_txt = '\n'.join(iprPy.tools.fill_template(data_info, info_dict, '<', '>'))
        
        #Generate column labels
        data_txt += '\n  %-16s' % 'r'
        for index, row in composition_data.iterrows():
            prototype = row.prototype
            if len(prototype) > 16:
                prototype = '--'.join(prototype.split('--')[:-1])
            data_txt += ' %-16s' % prototype
        data_txt += '\n'
        
        #Generate data rows 
        r_values = composition_data.iloc[0]['r (A)']
        for i in xrange(len(r_values)):
            data_txt += ' %16.10f' % r_values[i]
            for index, row in composition_data.iterrows():
                data_txt += ' %16.10f' % row['Ecoh (eV)'][i]
            data_txt += '\n'
        
        #Save data_txt
        with open(os.path.join(per_potential_directory, potential, 'EvsR.' + composition + '.txt'), 'w') as data_txt_file:
            data_txt_file.write(data_txt)
        

## 4. Plot generation

### 4.1 Parameters

Uses per_potential_directory generated in 3.1

### 4.2 Plot Generation Parameters

In [23]:
line_color = {
   #elemental
    'A1--Cu--fcc':                'black',
    'A2--W--bcc':                 'blue',
    'A3--Mg--hcp':                'red',
    'A3\'--alpha-La--double-hcp': 'cyan',
    'A4--C--dc':                  'magenta',
    'A5--beta-Sn':                '#EAC117',
    'A6--In--bct':                'orange',
    'A7--alpha-As':               'gray',
    'A15--beta-W':                'green',
    'Ah--alpha-Po--sc':           'brown',
   #1:1
    'B1--NaCl--rock-salt':        'black',
    'B2--CsCl':                   'blue',
    'B3--ZnS--cubic-zinc-blende': 'red',
    'L1_0--AuCu':                 'cyan',
   #1:2
    'C1--CaF2--fluorite':         'black',
   #1:3
    'A15--Cr3Si':                 'black',
    'D0_3--BiF3':                 'blue',
    'L1_2--AuCu3':                'red',
   #1:1:2
    'L2_1--AlCu2Mn--heusler':     'black'
}

In [24]:
line_style = {
   #elemental
    'A1--Cu--fcc':                'solid',
    'A2--W--bcc':                 'solid',
    'A3--Mg--hcp':                'dashed',
    'A3\'--alpha-La--double-hcp': '-.',
    'A4--C--dc':                  'solid',
    'A5--beta-Sn':                'solid',
    'A6--In--bct':                'solid',
    'A7--alpha-As':               'solid',
    'A15--beta-W':                'solid',
    'Ah--alpha-Po--sc':           'solid',
   #1:1
    'B1--NaCl--rock-salt':        'solid',
    'B2--CsCl':                   'solid',
    'B3--ZnS--cubic-zinc-blende': 'solid',
    'L1_0--AuCu':                 'solid',
   #1:2
    'C1--CaF2--fluorite':         'solid',
   #1:3
    'A15--Cr3Si':                 'solid',
    'D0_3--BiF3':                 'solid',
    'L1_2--AuCu3':                'solid',
   #1:1:2
    'L2_1--AlCu2Mn--heusler':     'solid'
}

### 4.3 Code

In [25]:
#Loop over all potentials
for potential in np.unique(data.potential):
    potential_data = data.loc[data.potential==potential]
    
    #Check that a directory exists for the potential
    if not os.path.isdir(os.path.join(per_potential_directory, potential)):
        os.makedirs(os.path.join(per_potential_directory, potential))
        
    #Loop over all compositions
    for composition in np.unique(potential_data.composition):
        composition_data = potential_data.loc[potential_data.composition==composition]
        
        #Prepare plot
        plt.title('Cohesive Energy vs. Interatomic Spacing for '+composition)
        plt.xlabel('r (A)')
        plt.ylabel('Cohesive Energy (eV/atom)')
        ymin = 0
        
        #Plot data 
        r_values = composition_data.iloc[0]['r (A)']
        for index, row in composition_data.iterrows():
            prototype = row.prototype
            plt.plot(r_values, row['Ecoh (eV)'], label=prototype,
                     color=line_color[prototype], linestyle=line_style[prototype],  linewidth=2)
            if min(row['Ecoh (eV)']) < ymin: ymin = min(row['Ecoh (eV)'])
        
        #Finish up plot
        if ymin < 0:
            plt.xlim(min(r_values), max(r_values))
            plt.ylim(floor(10.5*ymin)/10.,0)
            plt.legend(loc='lower right')
            plt.savefig(os.path.join(per_potential_directory, potential, 'EvsR.' + composition + '.png'))
        plt.close()

## 5. HTML Generation

In [32]:
html_info = """
<h2>Cohesive Energy vs. Interatomic Spacing</h2>
<p>
    Plots of the cohesive energy vs interatomic spacing, <i>r</i>, are shown below 
    for a number of crystal structures.  The structures are generated based on the 
    ideal atomic positions and <i>b</i>/<i>a</i> and <i>c</i>/<i>a</i> lattice 
    parameter ratios for a given crystal prototype. The size of the system is then 
    uniformly scaled, and the energy calculated without relaxing the system. To 
    obtain these plots, 300 values of <i>r</i> are evaluated between 2.0 and 6.0 
    &Aring;.  Clicking on the plot will open a table containing the numerical values.
</p><p>
    More information about the calculation used can be found on the 
    <a href="http://www.ctcms.nist.gov/potentials/tools.html">Tools</a> page.
</p><p>
    <a href="http://www.nist.gov/public_affairs/disclaimer.cfm">NIST disclaimer</a>
</p><p>
    <b>Disclaimer</b>: These results are meant to be used as guidelines for 
    comparing potentials, not as definitive measurements for the properties. The minima 
    identified by this calculation do not guarantee that the associated crystal 
    structures will be stable since no relaxation is performed. Variations in the values 
    may occur for fully relaxed configurations, different <i>b</i>/<i>a</i> and 
    <i>c</i>/<i>a</i> lattice parameter ratios, different simulation software and 
    different implementations of the interatomic potential. 
</p><p>
    <b>Version Information:</b> As property calculation methods are developed and updated, there 
    may be changes in the calculated values. Updates to the calculation methods 
    that affect the values will be documented and archival versions of this page 
    will be made available as a record. 
    <ul><li>
        2016-09-28. Plots for binary structures added. Data and plots for elemental 
        structures regenerated. Data values match the values of the previous version.
        Data table formatting slightly changed to increase precision and ensure spaces 
        between large values. Composition added to plot title and structure names made 
        longer. 
    </li><li>
        2016-04-07. Plots for elemental structures added. 
    </li></ul>
</p>
<hr/>
"""

In [33]:
#Loop over all potentials
for potential in np.unique(data.potential):
    potential_data = data.loc[data.potential==potential]
    
    #Start html with html_info
    html = html_info
        
    #Loop over all compositions
    for composition in np.unique(potential_data.composition):
        composition_data = potential_data.loc[data.composition==composition]
        
        #Append plot references
        html += '<h3>Cohesive Energy vs. Interatomic Spacing for ' + composition + '</h3>\n'
        html += '<a href="./' + potential + '/EvsR.' + composition + '.txt" target="_blank">'
        html += '<img src="./' + potential + '/EvsR.' + composition + '.png"></a>\n<hr/\n>'
        
    #Save html
    with open(os.path.join(per_potential_directory, potential, 'EvsR.info'), 'w') as html_file:
        html_file.write(html)

## 4. Comparison Plots